In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
application_train = pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
application_test = pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")

bureau = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau.csv")
bureau_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/bureau_balance.csv")

credit_card_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/credit_card_balance.csv")
installments_payments = pd.read_csv("/kaggle/input/home-credit-default-risk/installments_payments.csv")

POS_CASH_balance = pd.read_csv("/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv")
previous_application = pd.read_csv("/kaggle/input/home-credit-default-risk/previous_application.csv")

In [ ]:
dataframe_dict = {"application_train": application_train,
                  "application_test": application_test,
                  "bureau": bureau,
                  "bureau_balance" : bureau_balance,
                  "credit_card_balance": credit_card_balance,
                  "installments_payments": installments_payments,
                  "POS_CASH_balance": POS_CASH_balance,
                  "previous_application": previous_application               
    
}


Basic data checks


shape

In [ ]:
for key in dataframe_dict:
    print("Shape of {0} is {1}".format(key, dataframe_dict[key].shape))

In [ ]:
application_train.columns

Removing duplicates

In [ ]:
application_train.drop_duplicates(inplace= True)
bureau.drop_duplicates(inplace= True)
# bureau_balance.drop_duplicates(inplace= True)
# credit_card_balance.drop_duplicates(inplace= True)
# installments_payments.drop_duplicates(inplace= True)
# POS_CASH_balance.drop_duplicates(inplace= True)
previous_application.drop_duplicates(inplace= True)

Primary key check

In [ ]:
application_train.groupby(by=["SK_ID_CURR"]).size().reset_index(name="counts")['counts'].value_counts()

EDA

Application train

missing data

dropping all cols with <30% data

In [ ]:
application_train = application_train.replace({'':np.nan})
application_train = application_train.dropna(thresh=0.7*len(application_train), axis= 1)

In [ ]:
Miss = pd.DataFrame(columns=["Miss%"])
Miss["Miss%"]=((application_train.isnull().sum())*100/len(application_train))
Miss[Miss["Miss%"]>0]

Target class distribution

In [ ]:
application_train["TARGET"].value_counts()

In [ ]:
application_train["credit-income ratio"] =  application_train["AMT_CREDIT"] / application_train["AMT_INCOME_TOTAL"]

In [ ]:
application_train["credit-income ratio"].describe()

In [ ]:
plt.hist(application_train["credit-income ratio"],bins=100)
plt.show()

In [ ]:

application_train["Years_Employed"] =  application_train["DAYS_EMPLOYED"] / float(-365)
application_train.drop('DAYS_EMPLOYED', inplace=True, axis=1)

In [ ]:
application_train["Years_Employed"].describe()

In [ ]:
plt.hist(application_train["Years_Employed"],bins=100)
plt.show()

In [ ]:
sns.countplot(application_train['CODE_GENDER'],hue=application_train['TARGET'])

In [ ]:
sns.countplot(application_train['CNT_CHILDREN'],hue=application_train['TARGET'])

In [ ]:
application_train["Age_in_years"] = application_train['DAYS_BIRTH']/float(-365)
application_train.drop('DAYS_BIRTH', inplace=True, axis=1)
g = sns.FacetGrid(application_train, col='TARGET')
g.map(sns.distplot, 'Age_in_years')

In [ ]:
sns.countplot(application_train['TARGET'],hue=application_train['FLAG_OWN_CAR'])

In [ ]:
application_train.loc[(application_train['TARGET']==0)]["ORGANIZATION_TYPE"].value_counts()

In [ ]:

g = sns.FacetGrid(application_train, col='TARGET')
g.map(sns.distplot, 'EXT_SOURCE_2')

In [ ]:
application_train.columns

Bureau data

In [ ]:
bureau.head()

In [ ]:
bureau.isnull().sum()

In [ ]:
bureau = bureau.replace({'':np.nan})
bureau = bureau.dropna(thresh=0.7*len(bureau), axis= 1)

In [ ]:
bureau.shape

bureau_balance

In [ ]:
bureau_balance.head()

In [ ]:
bureau_balance.isnull().sum()

credit_card_balance

In [ ]:
credit_card_balance.head()

In [ ]:
credit_card_balance = credit_card_balance.replace({'':np.nan})
credit_card_balance = credit_card_balance.dropna(thresh=0.7*len(credit_card_balance), axis= 1)

In [ ]:
credit_card_balance.isnull().sum()

previous_application

In [ ]:
previous_application.head()

In [ ]:
previous_application = previous_application.replace({'':np.nan})
previous_application = previous_application.dropna(thresh=0.7*len(previous_application), axis= 1)

In [ ]:
previous_application.isnull().sum()

installments_payments

In [ ]:
installments_payments.head()

In [ ]:
installments_payments = installments_payments.replace({'':np.nan})
installments_payments = installments_payments.dropna(thresh=0.7*len(installments_payments), axis= 1)

In [ ]:
installments_payments.isnull().sum()

Data preparation

In [ ]:
app_train=application_train[["SK_ID_CURR","TARGET"]]

In [ ]:
app_train_bureau = app_train.merge(bureau,how='inner', on=['SK_ID_CURR'])
app_train_bureau_prev = app_train_bureau.merge(previous_application,how='inner', on=['SK_ID_CURR'])


In [ ]:
app_train_bureau_prev.isnull().sum()

In [ ]:
Miss_bureau_pre = pd.DataFrame(columns=["Miss%"])
Miss_bureau_pre["Miss%"]=((app_train_bureau_prev.isnull().sum())*100/len(app_train_bureau_prev))
Miss_bureau_pre[Miss_bureau_pre["Miss%"]>0]

In [ ]:
l= ["DAYS_CREDIT_ENDDATE","AMT_CREDIT_SUM","AMT_CREDIT_SUM_DEBT","AMT_ANNUITY","AMT_CREDIT","AMT_GOODS_PRICE",
    "CNT_PAYMENT","PRODUCT_COMBINATION"]
num_data_bureau_prev= app_train_bureau_prev[l].select_dtypes(include= [np.number])
cat_data_bureau_prev = app_train_bureau_prev[l].select_dtypes(exclude= [np.number])

col_miss_num_bureau_prev = list(num_data_bureau_prev.columns)
col_miss_cat_bureau_prev = list(cat_data_bureau_prev.columns)

In [ ]:
l1= ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'CNT_FAM_MEMBERS',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
       'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
       'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']


num_data_app= application_train[l1].select_dtypes(include= [np.number])
cat_data_app = application_train[l1].select_dtypes(exclude= [np.number])

col_miss_num_app= list(num_data_app.columns)
col_miss_cat_app = list(cat_data_app.columns)


In [ ]:
med_bureau_prev = app_train_bureau_prev[col_miss_num_bureau_prev].median()
med_app = application_train[col_miss_num_app].median()

mode_bureau_prev = app_train_bureau_prev[col_miss_cat_bureau_prev].mode()
mode_app = application_train[col_miss_cat_app].mode()

In [ ]:
Dict_val_bureau_prev = med_bureau_prev.to_dict()
Dict_val_app = med_app.to_dict()

Dict_val_cat_bureau_prev = {"PRODUCT_COMBINATION":"Cash"
    
}
Dict_val_cat_app = {"NAME_TYPE_SUITE":"Unaccompanied"}

In [ ]:
app_train_bureau_prev.fillna(value= Dict_val_bureau_prev, inplace=True)
application_train.fillna(value= Dict_val_app, inplace=True)

app_train_bureau_prev.fillna(value= Dict_val_cat_bureau_prev, inplace=True)
application_train.fillna(value= Dict_val_cat_app, inplace=True)



Correlation

In [ ]:
Cor= application_train.corr().abs()
col_app = Cor.nlargest(15,"TARGET").index
col_app

In [ ]:
Cor= app_train_bureau_prev.corr().abs()
col_bureau_prev = Cor.nlargest(10,"TARGET").index
col_bureau_prev

In [ ]:
app_train_bureau_prev.isnull().sum().value_counts()


In [ ]:
app_final = application_train[['SK_ID_CURR','TARGET', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'Age_in_years',
       'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT',
       'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH', 'REG_CITY_NOT_WORK_CITY',
       'FLAG_EMP_PHONE', 'Years_Employed', 'REG_CITY_NOT_LIVE_CITY',
       'FLAG_DOCUMENT_3', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE']]

bureau_final = bureau[["SK_ID_CURR",'DAYS_CREDIT', 'DAYS_CREDIT_UPDATE', 'DAYS_CREDIT_ENDDATE','AMT_CREDIT_SUM_OVERDUE']]
prev_final = previous_application [['SK_ID_CURR','CNT_PAYMENT','DAYS_DECISION','AMT_CREDIT', 'HOUR_APPR_PROCESS_START',
                                    'AMT_GOODS_PRICE',]]


In [ ]:
app_train_bureau_final = app_final.merge(bureau_final,how='inner', on=['SK_ID_CURR'])
app_train_bureau_prev_final = app_train_bureau_final.merge(prev_final,how='inner', on=['SK_ID_CURR'])


In [ ]:
del bureau_balance
del credit_card_balance
del installments_payments
del POS_CASH_balance
del app_train_bureau_prev
del num_data_bureau_prev
del cat_data_bureau_prev
del num_data_app
del cat_data_app
del app_train_bureau_final
del app_train
del app_train_bureau


In [ ]:
app_train_bureau_prev_final.fillna(value= Dict_val_bureau_prev, inplace=True)

In [ ]:
app_train_bureau_prev_final.drop('AMT_GOODS_PRICE_y', inplace=True, axis=1)
app_train_bureau_prev_final.columns


In [ ]:
app_train_bureau_prev_final.skew()

In [ ]:
app_train_bureau_prev_final["AMT_CREDIT_SUM_OVERDUE"]=np.log((0.01+app_train_bureau_prev_final["AMT_CREDIT_SUM_OVERDUE"]))

Modelling

In [ ]:
from sklearn.utils import resample

In [ ]:
df_train_final_majority = app_train_bureau_prev_final[app_train_bureau_prev_final.TARGET==0]
df_train_final_minority = app_train_bureau_prev_final[app_train_bureau_prev_final.TARGET==1]
 
# Upsample minority class
df_train_final_minority_upsampled = resample(df_train_final_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=7159298,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_train_final = pd.concat([df_train_final_majority, df_train_final_minority_upsampled])
del df_train_final_majority
del df_train_final_minority_upsampled
del df_train_final_minority
del app_train_bureau_prev_final
# Display new class counts
df_train_final.TARGET.value_counts()

In [ ]:
df_train_final.columns

In [ ]:

df_train_final["SK_ID_CURR"] = df_train_final["SK_ID_CURR"].astype("int32") 
df_train_final["TARGET"] = df_train_final["TARGET"].astype("int16") 
df_train_final["EXT_SOURCE_2"] = df_train_final["EXT_SOURCE_2"].astype("float32") 
df_train_final["EXT_SOURCE_3"] = df_train_final["EXT_SOURCE_3"].astype("float32") 
df_train_final["Age_in_years"] = df_train_final["Age_in_years"].astype("int16") 
df_train_final["REGION_RATING_CLIENT_W_CITY"] = df_train_final["REGION_RATING_CLIENT_W_CITY"].astype("int16") 
df_train_final["REGION_RATING_CLIENT"] = df_train_final["REGION_RATING_CLIENT"].astype("int16") 
df_train_final["DAYS_LAST_PHONE_CHANGE"] = df_train_final["DAYS_LAST_PHONE_CHANGE"].astype("int16") 
df_train_final["DAYS_ID_PUBLISH"] = df_train_final["DAYS_ID_PUBLISH"].astype("int32") 
df_train_final["REG_CITY_NOT_WORK_CITY"] = df_train_final["REG_CITY_NOT_WORK_CITY"].astype("int32") 
df_train_final["FLAG_EMP_PHONE"] = df_train_final["FLAG_EMP_PHONE"].astype("int16") 
df_train_final["Years_Employed"] = df_train_final["Years_Employed"].astype("int16") 
df_train_final["REG_CITY_NOT_LIVE_CITY"] = df_train_final["REG_CITY_NOT_LIVE_CITY"].astype("int32") 
df_train_final["FLAG_DOCUMENT_3"] = df_train_final["FLAG_DOCUMENT_3"].astype("int16") 
df_train_final["DAYS_REGISTRATION"] = df_train_final["DAYS_REGISTRATION"].astype("int32") 
df_train_final["AMT_GOODS_PRICE_x"] = df_train_final["AMT_GOODS_PRICE_x"].astype("float32") 
df_train_final["DAYS_CREDIT"] = df_train_final["TARGET"].astype("int16") 
df_train_final["DAYS_CREDIT_UPDATE"] = df_train_final["DAYS_CREDIT_UPDATE"].astype("float32") 
df_train_final["DAYS_CREDIT_ENDDATE"] = df_train_final["DAYS_CREDIT_ENDDATE"].astype("float32") 
df_train_final["AMT_CREDIT_SUM_OVERDUE"] = df_train_final["AMT_CREDIT_SUM_OVERDUE"].astype("int16") 
df_train_final["CNT_PAYMENT"] = df_train_final["CNT_PAYMENT"].astype("float32") 
df_train_final["DAYS_DECISION"] = df_train_final["DAYS_DECISION"].astype("int16") 
df_train_final["AMT_CREDIT"] = df_train_final["AMT_CREDIT"].astype("int32") 
df_train_final["HOUR_APPR_PROCESS_START"] = df_train_final["HOUR_APPR_PROCESS_START"].astype("int16") 

In [ ]:
from sklearn.model_selection import train_test_split
X=df_train_final[['SK_ID_CURR','EXT_SOURCE_2', 'EXT_SOURCE_3', 'Age_in_years',
       'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT',
       'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH', 'REG_CITY_NOT_WORK_CITY',
       'FLAG_EMP_PHONE', 'Years_Employed', 'REG_CITY_NOT_LIVE_CITY',
       'FLAG_DOCUMENT_3', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE_x']]
y=df_train_final['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

In [ ]:
from sklearn.metrics import roc_auc_score

# auc scores
# auc_score1 = roc_auc_score(y_test, predictions)
# accuracy_score(y_test, predictions)

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(C=0.001)
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)

auc_score = roc_auc_score(y_test, predictions)
auc_score

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
clf = RandomForestClassifier()  
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)
auc_score1 = roc_auc_score(y_test, y_pred)
auc_score1

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(learning_rate =0.1,
 n_estimators=200,
 max_depth=5,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 eval_metric = "auc",
 nthread=4,
 scale_pos_weight=1,
 seed=27)

model.fit(X_train,y_train)
pred= model.predict(X_test) 
auc_score2 = roc_auc_score(y_test, pred)
auc_score2

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

Predictions

In [ ]:
application_test["Age_in_years"] = application_test['DAYS_BIRTH']/float(-365)
application_test.drop('DAYS_BIRTH', inplace=True, axis=1)

application_test["Years_Employed"] =  application_test["DAYS_EMPLOYED"] / float(-365)
application_test.drop('DAYS_EMPLOYED', inplace=True, axis=1)


In [ ]:
app_test= application_test[['SK_ID_CURR','EXT_SOURCE_2', 'EXT_SOURCE_3', 'Age_in_years',
       'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT',
       'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH', 'REG_CITY_NOT_WORK_CITY',
       'FLAG_EMP_PHONE', 'Years_Employed', 'REG_CITY_NOT_LIVE_CITY',
       'FLAG_DOCUMENT_3', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE']]

In [ ]:
app_test_bureau_final = app_test.merge(bureau_final,how='inner', on=['SK_ID_CURR'])
app_test_bureau_prev_final = app_test_bureau_final.merge(prev_final,how='inner', on=['SK_ID_CURR'])

del app_test
del app_test_bureau_final

app_test_bureau_prev_final.fillna(value= Dict_val_app, inplace=True)
app_test_bureau_prev_final.fillna(value= Dict_val_bureau_prev, inplace=True)
app_test_bureau_prev_final.drop('AMT_GOODS_PRICE_y', inplace=True, axis=1)
app_test_bureau_prev_final["AMT_CREDIT_SUM_OVERDUE"]=np.log((0.01+app_test_bureau_prev_final["AMT_CREDIT_SUM_OVERDUE"]))



In [ ]:
app_test_bureau_prev_final=app_test_bureau_prev_final[['SK_ID_CURR','EXT_SOURCE_2', 'EXT_SOURCE_3', 'Age_in_years',
       'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT',
       'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH', 'REG_CITY_NOT_WORK_CITY',
       'FLAG_EMP_PHONE', 'Years_Employed', 'REG_CITY_NOT_LIVE_CITY',
       'FLAG_DOCUMENT_3', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE_x']]

In [ ]:

p=logisticRegr.predict(app_test_bureau_prev_final)

In [ ]:
final = app_test_bureau_prev_final[['SK_ID_CURR']]
final['TARGET'] = p

In [ ]:
final.head(25)

In [ ]:
final["TARGET"].value_counts()